# AT&T Spam Detector - BertForSequenceClassification - bert-base-uncased

## <span style="color:orange"><b>DOES'NT WORK</b></span>

This code use 
* BertForSequenceClassification
* bert-base-uncased (where cat & CAT are the same)


In [1]:
# prelude

import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input
from transformers import TFBertForSequenceClassification, BertTokenizer # 'TF' for TensorFlow models
from sklearn.model_selection import train_test_split
# from tensorflow.keras.optimizers import Adam

from pathlib import Path
k_Current_dir = Path.cwd()
k_AssetsDir = "assets"
k_sms_max_len   = 100
k_random_state  = 42
k_test_size     = 0.3
k_batch_size    = 32



c:\Users\phili\anaconda3\envs\tf_cpu_bert_seq\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# -----------------------------------------------------------------------------
# drop empty cols and duplicates, rename cols...
def cleaner(df):
    df.drop(columns="Unnamed: 2", inplace=True)
    df.drop(columns="Unnamed: 3", inplace=True)
    df.drop(columns="Unnamed: 4", inplace=True)

    df.drop_duplicates(inplace=True)

    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace("/", "_")

    df.rename(columns={"v1": "labels"}, inplace=True)
    df.rename(columns={"v2": "texts"}, inplace=True)

    df["labels"] = df["labels"].map({"ham": 0, "spam": 1})

    return df

In [6]:
df = pd.read_csv(k_Current_dir / k_AssetsDir / "spam.csv", encoding="cp1252")
df = cleaner(df)

labels = df['labels'].tolist()
texts = df['texts'].tolist()

input_ids       = Input(shape=(k_sms_max_len,), dtype=tf.int32, name="input_ids")
attention_masks = Input(shape=(k_sms_max_len,), dtype=tf.int32, name="attention_mask")

model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # modèle TensorFlow

# ! Fige toutes les couches, y compris la tête de classification, ce qui rend tout le modèle non entraînable.
# Pas une bonne idée
# for layer in model.layers:

# Ne fige que les couches internes de BERT. Permet d'entraîner la tête de classification
# Plus mailin
for layer in model.bert.encoder.layer:
    layer.trainable = False

model.summary()

path = Path(f"{k_Current_dir/k_AssetsDir/'bert_seq_uncased_02_arch.png'}")
tf.keras.utils.plot_model(model, path, show_shapes=True)


# bert-base-uncased : cat & CAT are equal
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
encoded_data = tokenizer(
    texts,
    max_length=k_sms_max_len,
    padding='max_length',       # sequences will be padded according the value of the parameter max_length
    truncation=True,
    return_tensors='tf'         # "tf" for TensorFlow
)

X_train_ids, X_test_ids, X_train_mask, X_test_mask, y_train, y_test = train_test_split(
    encoded_data['input_ids'].numpy(), 
    encoded_data['attention_mask'].numpy(), 
    labels, 
    test_size=k_test_size, 
    random_state=k_random_state
)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=2)

# convert the datasets into tensors
X_train_ids     = tf.convert_to_tensor(X_train_ids)
X_test_ids      = tf.convert_to_tensor(X_test_ids)
X_train_mask    = tf.convert_to_tensor(X_train_mask)
X_test_mask     = tf.convert_to_tensor(X_test_mask)
y_train         = tf.convert_to_tensor(y_train)
y_test          = tf.convert_to_tensor(y_test)

# Gather encoded data into dictionaries for training
X_train = {'input_ids': X_train_ids, 'attention_mask': X_train_mask}
X_test = {'input_ids': X_test_ids, 'attention_mask': X_test_mask}



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_113 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 24,429,314
Non-trainable params: 85,054,464
_________________________________________________________________


In [7]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08), 
    # loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    # metrics=[tf.keras.metrics.Recall(name="recall"), tf.keras.metrics.Precision(name="precision"), "accuracy"],       # name=... avoid recall_1 for example
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')],
)

history = model.fit(
    [X_train['input_ids'], X_train['attention_mask']],
    y_train,
    batch_size = k_batch_size,
    epochs = 2,
    validation_data=([X_test['input_ids'], X_test['attention_mask']], y_test),
)


Epoch 1/2


ValueError: in user code:

    File "c:\Users\phili\anaconda3\envs\tf_cpu_bert_seq\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\phili\anaconda3\envs\tf_cpu_bert_seq\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\phili\anaconda3\envs\tf_cpu_bert_seq\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\phili\anaconda3\envs\tf_cpu_bert_seq\lib\site-packages\transformers\modeling_tf_utils.py", line 922, in train_step
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "c:\Users\phili\anaconda3\envs\tf_cpu_bert_seq\lib\site-packages\keras\engine\compile_utils.py", line 459, in update_state
        metric_obj.update_state(y_t, y_p, sample_weight=mask)
    File "c:\Users\phili